In [ ]:
!pip install -q trino pandas requests plotly dash-bootstrap-templates

In [ ]:
from trino.dbapi import connect
from trino.auth import OAuth2Authentication, JWTAuthentication
from trino.sqlalchemy import URL
from sqlalchemy import create_engine
import pandas as pd
import requests
import base64
import pandas as pd
from trino.exceptions import TrinoUserError
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash_bootstrap_templates import load_figure_template
load_figure_template('solar')

color_palette = ['#004595', '#4287f5', '#1EB3C0', '#F5B31E']
pio.templates['solar']['layout']['font']['color']='white'

TRINO_URI = "https://trino.dl.amunanalytics.eu"


In [ ]:
# some utility functions to get an access token from the Amun Analytics Oauth
AUTHENTIK_TOKEN_ENDPOINT = "https://sso.amunanalytics.eu/application/o/token/"

def get_client_secret(user, pwd):
    return base64.b64encode(f"{user}:{pwd}".encode()).decode()

def get_access_token(user, pwd):
    client_secret = get_client_secret(user, pwd)
    scope = "lakekeeper"

    r = requests.post(
        AUTHENTIK_TOKEN_ENDPOINT,
        data={
            "grant_type": "client_credentials",
            "client_id": 'GU9V8DlBxgpf5nhtdPrW174h3UUzQn8RLPaxFf7a',
            "client_secret": client_secret,
            "scope": scope
        }
    )
    r.raise_for_status()
    return r.json()['access_token']

In [ ]:
pd.options.plotting.backend = "plotly"
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 10)
pd.set_option('display.max_columns', 30)

In [ ]:
# define the connection to oauth, fill in your username and token below
conn = connect(
    host=TRINO_URI,
    catalog="lakekeeper",
    auth=JWTAuthentication(
        get_access_token(
            user='INSERT_YOUR_USERNAME_HERE',
            pwd='INSERT_YOUR_TOKEN_HERE'
        )
    ),
)
cur = conn.cursor()

In [ ]:
# Query the datalake through trino with standard SQL, in thise case we retrieve all entries for a specific CNEC
# to see how far your query is you can visit https://trino.dl.amunanalytics.eu/ and login through the Single Sign On
df = pd.read_sql("""
select *
from flowbased_dayahead.final_domain
where cne_name = 'Altheim - Sittling 219'
and cont_name = 'Altheim - Sittling 220'
and business_day >= '2025-08-01'
and business_day < '2025-12-31 23:59'
""", conn)

In [ ]:
# the datalake saves all timestamps in UTC, for visualization we want to view it in local time, this line converts it for us
df['mtu'] = df['mtu'].dt.tz_localize('UTC').dt.tz_convert('europe/amsterdam')

In [ ]:
df

In [ ]:
# for each CNEC per mtu there are 2 versions for each direction. take the worst case per mtu here
S=df.groupby('mtu')['ram_pct'].min()

In [ ]:
# create a nice figure of those worst case RAM over time
fig_ram = S.resample('d').mean().plot()\
.update_layout(showlegend=False, title='Average RAM per day')\
.update_yaxes(range=(200, 400), title='RAM [MW]')
fig_ram

In [ ]:
# calculate the zone to zone PTDF
df['maxz2z']=df[[c for c in df if c.startswith('ptdf_')]].max(axis=1)-df[[c for c in df if c.startswith('ptdf_')]].min(axis=1)

In [ ]:
# take worst case max z2z PTDF per MTU
S2=df.groupby('mtu')['maxz2z'].max()*100

In [ ]:
# create a nice figure of averaged max z2z (averaged per day, with each mtu having the worst case)
fig_z2z = S2.resample('d').mean().plot()\
.update_layout(showlegend=False, title='Average max Z2Z per day')\
.update_yaxes(title='max Z2Z [PTDF %]', range=(5,10))
fig_z2z

In [ ]:
# query for the same cnec when it was an active constraint in market coupling
df_ac = pd.read_sql("""
select * from flowbased_dayahead.active_constraints
where cne_eic = '11TD2L000000003L'
and cont_name = 'N-1 Altheim - Sittling 220'
and business_day >= '2025-08-01'
and business_day < '2025-12-31 23:59'
""",conn)

In [ ]:
# again fix timezone
df_ac['mtu'] = df_ac['mtu'].dt.tz_localize('UTC').dt.tz_convert('europe/amsterdam')

In [ ]:
df_ac = df_ac.sort_values('mtu')

In [ ]:
# calculate some statistics: average shadow price per day and number of MTU's with at least one active constraint of the one selected
df_ac_stats = pd.DataFrame({
    'average shadowprice': df_ac.groupby('business_day')['shadow_price'].mean(),
    'number of unique hours': df_ac.groupby('business_day')['mtu'].nunique(),
})

In [ ]:
# put these statistics in a nice graph with dual y-axis
fig_shadow_prices = make_subplots(specs=[[{"secondary_y": True}]])

fig_shadow_prices.add_trace(
    go.Bar(
        x=df_ac_stats.index,
        y=df_ac_stats['average shadowprice'],
        name="average shadowprice"
    ),
    secondary_y=False,
)

fig_shadow_prices.add_trace(
    go.Scatter(
        x=df_ac_stats.index,
        y=df_ac_stats['number of unique hours'],
        name="number of unique hours",
        mode='markers',
        marker=dict(size=7, color='red')
    ),
    secondary_y=True,
)

fig_shadow_prices.update_layout(
    title_text="Average shadow price and number of MTU's per day",
    margin=dict(t=60, r=30),
    legend=dict(x=0.01, y=0.99)
)

fig_shadow_prices.update_yaxes(title_text="[EUR]", secondary_y=False)
fig_shadow_prices.update_yaxes(title_text="[# MTU's]", secondary_y=True)
fig_shadow_prices